# Trabalho Prático 2 – Aprendizado Não Supervisionado  
## Segmentação de Clientes com Algoritmos de Clusterização

### Equipe:
* Francisco Djalma Pereira da Silva Júnior - 554222
* Francisco Leudes Bezerra Neto - 552478
* Pablo Vinícius da Silva Araújo - 574229

### Objetivo:
O objetivo principal deste projeto é aplicar, comparar e avaliar o desempenho de três algoritmos de clusterização distintos a um conjunto de dados real. O processo envolve todas as etapas de um projeto de ciência de dados, desde a exploração e pré-processamento dos dados até a aplicação dos modelos e a interpretação dos resultados. Ao final, buscamos identificar segmentos (clusters) de clientes com perfis e comportamentos semelhantes, que possam ser utilizados para direcionar estratégias de marketing mais eficazes.

### Descrição do Conjunto de Dados:
Para este trabalho, foi selecionado o dataset **"Customer Personality Analysis"**, disponível publicamente na plataforma Kaggle.
* **Fonte:** [Kaggle - Customer Personality Analysis](https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis)
* **Amostras:** O dataset original contém **2.240** registros de clientes.
* **Características:** Possui **29** atributos (features) que detalham o perfil de cada consumidor em quatro categorias principais:
    * **Pessoas:** Dados demográficos como ano de nascimento, educação, estado civil e renda.
    * **Produtos:** Gastos em diferentes categorias de produtos (vinhos, carnes, frutas, etc.).
    * **Promoções:** Engajamento dos clientes com campanhas de marketing anteriores.
    * **Lugar:** Canais de compra utilizados pelos clientes (loja física, site, catálogo).

Este conjunto de dados é ideal para a tarefa de clusterização, pois não possui rótulos predefinidos (variável alvo), característica típica de problemas de aprendizado não supervisionado. A riqueza de atributos comportamentais e demográficos torna possível a identificação de segmentos relevantes.

### Algoritmos Utilizados

- **K-Means:** Rápido e eficiente para grandes datasets; assume clusters esféricos e de tamanho similar.
- **Hierárquico (Aglomerativo):** Permite análise mais interpretável via dendrograma; não requer definição prévia do número de clusters.
- **DBSCAN:** Identifica clusters de formatos arbitrários e detecta outliers naturalmente.

### Pipeline do Projeto
1. Importação e visualização inicial dos dados
2. Limpeza de dados e engenharia de atributos
3. Análise exploratória e identificação de outliers
4. Pré-processamento (normalização e codificação)
5. Aplicação dos algoritmos de clusterização
6. Avaliação e comparação dos modelos
7. Visualização com PCA e t-SNE
8. Perfilamento e interpretação dos clusters

### Divisão de Tarefas:

| Etapa                                          | Djalma | Leudes | Pablo |
|------------------------------------------------|:------:|:------:|:-----:|
| Escolha do dataset e definição do problema     |   ✔    |   ✔    |   ✔   |
| Importação e visualização inicial dos dados    |        |   ✔    |       |
| Limpeza e engenharia de features               |   ✔    |        |       |
| Análise exploratória e remoção de outliers     |        |        |   ✔   |
| Pré-processamento (encoding e scaling)         |   ✔    |        |   ✔   |
| Modelagem com K-Means                          |   ✔    |        |       |
| Modelagem com Clusterização Hierárquica        |        |   ✔    |       |
| Modelagem com DBSCAN                           |        |        |   ✔   |
| Avaliação dos modelos                          |   ✔    |   ✔    |   ✔   |
| Visualização dos clusters (PCA e t-SNE)        |   ✔    |        |       |
| Conclusão e perfilamento                       |   ✔    |   ✔    |   ✔   |


# Importação de bibliotecas


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors

# Leitura do dataset

In [ ]:
df = pd.read_csv('marketing_campaign.csv', sep='\t')

# Informações gerais do dataset

In [ ]:
print("--- Informações Gerais e Valores Nulos ---")
df.info()

In [ ]:
# 2. Obter estatísticas descritivas para as colunas numéricas
print("\n\n--- Estatísticas Descritivas (Numéricas) ---")
# Usamos o .T para transpor a tabela e facilitar a leitura
display(df.describe().T)

In [ ]:
# 3. Verificar especificamente a quantidade de valores nulos por coluna
print("\n\n--- Contagem de Valores Nulos por Coluna ---")
print(df.isnull().sum())

In [ ]:
df.head()

# Data Cleaning

In [ ]:
df_cleaned = df.copy()

In [ ]:
median_income = df_cleaned['Income'].median()
df_cleaned['Income'] = df_cleaned['Income'].fillna(median_income)

In [ ]:
df_cleaned['Age'] = 2025 - df_cleaned['Year_Birth']

In [ ]:
mnt_cols = [col for col in df_cleaned.columns if 'Mnt' in col]
df_cleaned['Total_Spent'] = df_cleaned[mnt_cols].sum(axis=1)

In [ ]:
df_cleaned['Dt_Customer'] = pd.to_datetime(df_cleaned['Dt_Customer'], dayfirst=True)
today = pd.to_datetime('2025-07-04')
df_cleaned['Customer_Tenure'] = (today - df_cleaned['Dt_Customer']).dt.days

In [ ]:
df_cleaned['Living_With'] = df_cleaned['Marital_Status'] = df_cleaned['Marital_Status'].replace({
    'Married': 'In_Relationship',
    'Together': 'In_Relationship',
    'Single': 'Single',
    'Divorced': 'Single',
    'Widow': 'Single',
    'Alone': 'Single',
    'Absurd': 'Single',
    'YOLO': 'Single'
})

In [ ]:
df_cleaned['Children'] = df_cleaned['Kidhome'] + df_cleaned['Teenhome']

In [ ]:
df_cleaned['Family_Size'] = df_cleaned['Living_With'].map({
    'Single': 1,
    'In_Relationship': 2
}) + df_cleaned['Children']

In [ ]:
df_cleaned['Is_Parent'] = np.where(df_cleaned['Children'] > 0, 1, 0)

In [ ]:
df_cleaned['Education'] = df_cleaned['Education'].replace({
  'Basic': 'Undergraduate',
  '2n Cycle': 'Undergraduate',
  'Graduation': 'Graduate',
  'Master': 'Postgraduate',
  'PhD': 'Postgraduate'
})

In [ ]:
df_cleaned = df_cleaned.rename(
  columns= {
    'MntWines': 'Wine_Spent',
    'MntFruits': 'Fruit_Spent',
    'MntMeatProducts': 'Meat_Spent',
    'MntFishProducts': 'Fish_Spent',
    'MntSweetProducts': 'Sweet_Spent',
    'MntGoldProds': 'Gold_Spent',
  }
)

In [ ]:
to_drop = [
  'ID', 'Year_Birth', 'Dt_Customer', 'Z_CostContact', 'Z_Revenue', 'Marital_Status'
]
df_cleaned.drop(columns=to_drop, axis=1, inplace=True)

# Análise detalhada do dataset

In [ ]:
df_cleaned.info()

In [ ]:
print("--- Contagem por Status de Relacionamento (Após Limpeza) ---")
print(df_cleaned["Marital_Status"].value_counts())

print("\n\n--- Contagem por Nível de Escolaridade ---")
print(df_cleaned["Education"].value_counts())

In [ ]:
# --- PASSO DE ANÁLISE EXTRA: PAIRPLOT ---

print("Gerando o Pairplot... Este gráfico pode demorar um pouco para ser criado.")

# 1. Definir as colunas que queremos visualizar
# Usamos as nossas colunas: Total_Spent e Customer_Tenure
cols_for_pairplot = ["Income", "Recency", "Customer_Tenure", "Age", "Total_Spent", "Is_Parent"]

# 2. Criar o Pairplot
# Usamos o df_cleaned, antes da remoção de outliers, para ter a visão completa
sns.pairplot(
    df_cleaned[cols_for_pairplot], 
    hue="Is_Parent", 
    palette=["#682F2F", "#F3AB60"] # Usando a paleta de cores sugerida
)

# Adicionar um título geral
plt.suptitle('Relação Entre Features-Chave, Separado por Paternidade', y=1.02, fontsize=16)

# Exibir o gráfico
plt.show()

In [ ]:
# Configurando o estilo e a paleta de cores
sns.set_style("whitegrid")
pallet = ["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"]

In [ ]:
# --- Capítulo 1: Quem são os Nossos Clientes? (Análise Demográfica) ---
print("Capítulo 1: Perfil Demográfico dos Clientes")
fig1, axes1 = plt.subplots(2, 2, figsize=(18, 12))
fig1.suptitle('Capítulo 1: Quem são os Nossos Clientes?', fontsize=20, y=1.03)

# Gráfico 1.1: Distribuição da Renda
sns.histplot(ax=axes1[0, 0], data=df_cleaned, x='Income', kde=True, color=pallet[0], bins=30)
axes1[0, 0].set_title('Distribuição da Renda Anual', fontsize=14)

# Gráfico 1.2: Distribuição da Idade
sns.histplot(ax=axes1[0, 1], data=df_cleaned, x='Age', kde=True, color=pallet[1], bins=30)
axes1[0, 1].set_title('Distribuição da Idade', fontsize=14)

# Gráfico 1.3: Nível de Escolaridade
sns.countplot(ax=axes1[1, 0], data=df_cleaned, x='Education', order=df_cleaned['Education'].value_counts().index, hue='Education', legend=False)
axes1[1, 0].set_title('Nível de Escolaridade', fontsize=14)

# Gráfico 1.4: Tamanho da Família
sns.countplot(ax=axes1[1, 1], data=df_cleaned, x='Family_Size', hue='Family_Size', legend=False)
axes1[1, 1].set_title('Tamanho da Família (Adultos + Crianças)', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# --- Capítulo 2: Como Eles Compram? (Análise de Consumo) ---
print("\nCapítulo 2: Comportamento de Consumo dos Clientes")
# Calculando o gasto total por categoria
spent_cols = ['Wine_Spent', 'Fruit_Spent', 'Meat_Spent', 'Fish_Spent', 'Sweet_Spent', 'Gold_Spent']
total_spent_by_cat = df_cleaned[spent_cols].sum().sort_values(ascending=False)

fig2, axes2 = plt.subplots(1, 2, figsize=(18, 7))
fig2.suptitle('Capítulo 2: Como Eles Compram?', fontsize=20, y=1.03)

# Gráfico 2.1: Gasto Total por Categoria de Produto
sns.barplot(ax=axes2[0], x=total_spent_by_cat.index, y=total_spent_by_cat.values, palette=pallet)
axes2[0].set_title('Gasto Total por Categoria de Produto', fontsize=14)
axes2[0].set_ylabel('Gasto Total')
axes2[0].tick_params(axis='x', rotation=45)

# Gráfico 2.2: Distribuição do Gasto Total por Cliente
sns.histplot(ax=axes2[1], data=df_cleaned, x='Total_Spent', kde=True, color=pallet[5], bins=40)
axes2[1].set_title('Distribuição do Gasto Total por Cliente', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# --- Capítulo 3: Quais Relações Existem nos Dados? (Análise de Correlação) ---
print("\nCapítulo 3: Relações Entre as Variáveis")
fig3, axes3 = plt.subplots(1, 2, figsize=(18, 7))
fig3.suptitle('Capítulo 3: Quais Relações Existem nos Dados?', fontsize=20, y=1.03)

# Gráfico 3.1: Relação entre Renda e Gasto Total
sns.scatterplot(ax=axes3[0], data=df_cleaned, x='Income', y='Total_Spent', hue='Is_Parent', palette="viridis")
axes3[0].set_title('Renda vs. Gasto Total (Colorido por "É Pai?")', fontsize=14)

# Gráfico 3.2: Heatmap de Correlação (com menos variáveis para ser mais legível)
cols_for_corr = ['Income', 'Age', 'Customer_Tenure', 'Total_Spent', 'Family_Size', 'Children']
corr = df_cleaned[cols_for_corr].corr()
sns.heatmap(ax=axes3[1], data=corr, annot=True, fmt=".2f", cmap='coolwarm')
axes3[1].set_title('Heatmap de Correlação das Principais Features', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(18, 12))
for i, col in enumerate(num_cols):
    plt.subplot(4, 3, i + 1)
    sns.boxplot(x=df_cleaned[col], color='lightcoral')
    plt.title(f'Boxplot de {col}')
plt.tight_layout()
plt.show()


In [ ]:
# Marital Status já foi agrupado em Living_With
sns.countplot(data=df_cleaned, x='Living_With', palette='Set2')
plt.title("Distribuição de Estado Civil (Agrupado)")
plt.show()

sns.countplot(data=df_cleaned, x='Education', palette='Set3', order=df_cleaned['Education'].value_counts().index)
plt.title("Distribuição de Nível de Educação")
plt.xticks(rotation=45)
plt.show()

sns.countplot(data=df_cleaned, x='Is_Parent', palette='Set1')
plt.title("Distribuição de Clientes com ou sem Filhos")
plt.xticks([0, 1], ["Sem filhos", "Com filhos"])
plt.show()


In [ ]:
# Seleciona apenas colunas numéricas
df_corr = df_cleaned.select_dtypes(include=np.number)

plt.figure(figsize=(14, 10))
sns.heatmap(df_corr.corr(), annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title("Mapa de Correlação das Variáveis Numéricas")
plt.show()

In [ ]:
# Gasto total vs. renda
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_cleaned, x='Income', y='Total_Spent', hue='Is_Parent', palette='Set1')
plt.title('Gasto Total vs. Renda')
plt.show()

# Idade vs. gasto com vinho
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_cleaned, x='Age', y='Wine_Spent', hue='Education', palette='Set2')
plt.title('Idade vs. Gasto com Vinho')
plt.show()


In [ ]:
# Criando a figura para o nosso dashboard de plots
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('Análise Visual de Features Importantes', fontsize=20, y=1.03)

# Gráfico 1: Boxplot da Renda (Income)
sns.boxplot(ax=axes[0, 0], x=df_cleaned['Income'], color=pallet[0])
axes[0, 0].set_title('Distribuição da Renda (Income)', fontsize=14)

# Gráfico 2: Boxplot do Gasto Total (Total_Spent)
sns.boxplot(ax=axes[0, 1], x=df_cleaned['Total_Spent'], color=pallet[1])
axes[0, 1].set_title('Distribuição do Gasto Total (Total_Spent)', fontsize=14)

# Gráfico 3: Relação entre Renda e Gasto Total
sns.scatterplot(ax=axes[1, 0], data=df_cleaned, x='Income', y='Total_Spent', color=pallet[2])
axes[1, 0].set_title('Renda vs. Gasto Total', fontsize=14)

# Gráfico 4: Boxplot da Idade (Age)
sns.boxplot(ax=axes[1, 1], x=df_cleaned['Age'], color=pallet[3])
axes[1, 1].set_title('Distribuição da Idade (Age)', fontsize=14)

plt.tight_layout()
plt.show()